In [1]:
pip install selenium

     |████████████████████████████████| 904 kB 3.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pyvirtualdisplay

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install webdriver-manager


Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install pandas

     |████████████████████████████████| 9.7 MB 930 kB/s eta 0:00:01
     |████████████████████████████████| 15.4 MB 20 kB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install plotly

     |████████████████████████████████| 13.2 MB 700 kB/s eta 0:00:01
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11429 sha256=9d7b23c833f5f4233ce5e0eef7e8e89629f8008cb93c53e7d24e9d3160b111db
  Stored in directory: /home/yann/.cache/pip/wheels/c4/a7/48/0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying
Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install twilio

     |████████████████████████████████| 479 kB 534 kB/s eta 0:00:01
  Created wheel for twilio: filename=twilio-6.59.1-py2.py3-none-any.whl size=1268926 sha256=8fab1586eeb7ddde3e6dc3b1b36c5763c39060fc5cb9830e1e883f4c8705fb33
  Stored in directory: /home/yann/.cache/pip/wheels/b9/da/74/cdf7bc6c109c2ab191ef0ace7a6e1929157a978ac9aad770c5
Successfully built twilio
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import pandas as pd
import io
import plotly.graph_objects as go
import datetime
import os
import time
import sys


In [2]:
url = 'http://www.isere.gouv.fr/booking/create/14544'
params ={'startDate':'2015-01-02',
         'token':'61c063a2b206db9352c078f44e0a57b365b74e53',
         'format':'csv',
        }

r = requests.get(url)
print(r.content)

b'<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">\n<html><head>\n<title>403 Forbidden</title>\n</head><body>\n<h1>Forbidden</h1>\n<p>You don\'t have permission to access /booking/create/14544\non this server.</p>\n</body></html>\n'


In [9]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options
from datetime import datetime
from twilio.rest import Client

def sendMessage(message, recipent):
    # client credentials are read from TWILIO_ACCOUNT_SID and AUTH_TOKEN
    account_sid = "AC00e983f008a2aefe3c1996c91c863***"
    # Your Auth Token from twilio.com/console
    auth_token  = "cb0fb39410a2f57ad1eb24d9848d1***"

    client = Client(account_sid, auth_token)

    # this is the Twilio sandbox testing number
    from_whatsapp_number='whatsapp:+1415523888*'
    # replace this number with your own WhatsApp Messaging number
    to_whatsapp_number=recipent

    client.messages.create(body=message,
                           from_=from_whatsapp_number,
                           to=to_whatsapp_number)


def sendMessageAll(message):
    recipentsList = ["whatsapp:+33687482780", "whatsapp:+33767235335"]
    for recipent in recipentsList:
        sendMessage(message, recipent)
    
WINDOW_SIZE = "1920,1080"

chrome_options = Options()  
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)

testsNumber = 0;

#options = ["planning35227", "planning35228", "planning35229"]
options = ["planning35227", "planning35228"]
notFound = True;
while notFound:
    #time.sleep(30)
    for option in options:
        try:
            if testsNumber % 10 == 0:
                sendMessageAll('Still runing, tried ' + str(testsNumber) + ' times...')

            testsNumber = testsNumber + 1;

            if not notFound:
                continue
            time.sleep(20)

            now = datetime.now()
            # dd/mm/YY H:M:S
            dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
            print("Trial number " + str(testsNumber) + ", time =", dt_string)

            driver = webdriver.Chrome(ChromeDriverManager().install())
            try: 
                #driver = webdriver.Chrome(options=chrome_options)
                driver.get("http://www.isere.gouv.fr/booking/create/14544")
            except: 
                print("Fail to load page")
                driver.close()
                time.sleep(600)
                continue
            time.sleep(1)
            try:
                driver.find_element_by_id("condition").click()
                time.sleep(1)
                driver.find_element(By.NAME, "nextButton").click()
            except: 
                print("Fail on 1st page")
                driver.close()
                time.sleep(600)
                continue
            time.sleep(1)
            try:
                optionEl = driver.find_element_by_id(option)
                optionEl.click();
                time.sleep(1)
                driver.find_element(By.NAME, "nextButton").click()
            except:
                print("Fail on 2nd page")
                driver.close()
                time.sleep(600)
                continue


            if "Il n'existe plus de plage horaire libre pour votre demande de rendez-vous" in driver.page_source:
                print("Plus de place option: " + option)
            elif "502" in driver.page_source:
                print("Bad gateway: " + option)
            elif "Site indisponible" in driver.page_source:
                print("Site indisponible: " + option)
            else:
                print("Something different! : " + option)
                message = 'Something different detected at http://www.isere.gouv.fr/booking/create/14544 \
                for option ' + option + '\n \
                \n \
                planning35227: matin \n \
                planning35228: apres midi \n \
                planning35229: mardis et vendredi après midi'
                sendMessageAll(message)
                notFound = False;
            if notFound:
                driver.close()
        except:
            print("Unknown fail")
            sendMessageAll("USorry princess Tingting, an error occured, but I ll reboot :)")

        



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/home/yann/.wdm/drivers/chromedriver/linux64/91.0.4472.19/chromedriver] found in cache


Trial number 1, time = 11/06/2021 08:49:46
Plus de place option: planning35227




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/home/yann/.wdm/drivers/chromedriver/linux64/91.0.4472.19/chromedriver] found in cache


Trial number 2, time = 11/06/2021 08:51:39
Plus de place option: planning35228




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/home/yann/.wdm/drivers/chromedriver/linux64/91.0.4472.19/chromedriver] found in cache


...

Trial number 22, time = 11/06/2021 10:07:36
Plus de place option: planning35228




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/home/yann/.wdm/drivers/chromedriver/linux64/91.0.4472.19/chromedriver] found in cache


Trial number 23, time = 11/06/2021 10:08:08
Plus de place option: planning35227




====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/home/yann/.wdm/drivers/chromedriver/linux64/91.0.4472.19/chromedriver] found in cache


Trial number 24, time = 11/06/2021 10:08:42
Something different! : planning35228


env: TWILIO_ACCOUNT_SID='AC00e983f008a2aefe3c1996c91c863203' # paste in Account SID between single quotes
env: TWILIO_AUTH_TOKEN='4dc084c7ccf70684ae2c87e2d47e5071' # paste Auth Token between single quotes


<Twilio.Api.V2010.MessageInstance account_sid=AC00e983f008a2aefe3c1996c91c863203 sid=SM2d79b38a23794f239f175084ecbc7275>